In [0]:
# Part 1 
# 2. read the files from the Azure storage and make a copy of it into DBFS. 

In [0]:
storage_account_name = "at2"
storage_account_access_key = "EnterYourAccessKeyHere"
blob_container_name = "at2"

In [0]:
dbutils.fs.mount(
  source = f'wasbs://{blob_container_name}@{storage_account_name}.blob.core.windows.net',
  mount_point = f'/mnt/{blob_container_name}/',
  extra_configs = {'fs.azure.account.key.' + storage_account_name + '.blob.core.windows.net': storage_account_access_key}
)

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-3252437749638083>, line 1
----> 1 dbutils.fs.mount(
      2   source = f'wasbs://{blob_container_name}@{storage_account_name}.blob.core.windows.net',
      3   mount_point = f'/mnt/{blob_container_name}/',
      4   extra_configs = {'fs.azure.account.key.' + storage_account_name + '.blob.core.windows.net': storage_account_access_key}
      5 )

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    360 exc.__context__ = None
    361 exc.__cause__ = None
--> 362 raise exc

ExecutionError: An error occurred while calling o886.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/at2; nested exception is: 
	java.lang.IllegalArgumentException: requirement 

In [0]:
dbutils.fs.ls("/mnt/at2")

In [0]:
dbutils.fs.cp("/mnt/at2", "/dbfs/FileStore", recurse=True)

In [0]:
# 3. Count the total numbers of rows for each taxi colour (yellow and green) by reading the files stored on DBFS
# Read Parquet files for yellow taxis
yellow_df = spark.read.parquet("dbfs:/dbfs/FileStore/" +  "yellow_taxi_*.parquet")

# Read Parquet files for green taxis
green_df = spark.read.parquet("dbfs:/dbfs/FileStore/" +  "green_taxi_*.parquet")

In [0]:
# count number of row
from pyspark.sql.functions import count
yellow_count = yellow_df.count()
green_count = green_df.count()
print("Yellow taxi count:", yellow_count)
print("Green taxi count:", green_count)

Yellow taxi count: 663055251
Green taxi count: 66200401


In [0]:
# load the location referential csv
taxi_zone_lookup = spark.read.option("header", True).csv("dbfs:/FileStore/taxi_zone_lookup-1.csv")

In [0]:
taxi_zone_lookup.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [0]:
# 5. Convert the “Green” 2015 parquet into a csv file
green_taxi_2015 = spark.read.option("header", True).parquet("/dbfs/FileStore/green_taxi_2015.parquet")
green_taxi_2015.write.option("header", True).csv( "/mnt/at2/green_taxi_2015.csv")

In [0]:
# 6. Data cleaning
yellow_df_clean = yellow_df
green_df_clean = green_df

In [0]:
yellow_df_clean.write.parquet("dbfs:/FileStore/yellow_df.parquet", mode="overwrite")
green_df_clean.write.parquet("dbfs:/FileStore/green_df.parquet", mode="overwrite")
yellow_df_clean = spark.read.parquet("dbfs:/FileStore/yellow_df.parquet")
green_df_clean = spark.read.parquet("dbfs:/FileStore/green_df.parquet")
yellow_df_clean.cache()
green_df_clean.cache()

DataFrame[VendorID: bigint, lpep_pickup_datetime: timestamp, lpep_dropoff_datetime: timestamp, store_and_fwd_flag: string, RatecodeID: double, PULocationID: bigint, DOLocationID: bigint, passenger_count: double, trip_distance: double, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, ehail_fee: double, improvement_surcharge: double, total_amount: double, payment_type: double, trip_type: double, congestion_surcharge: double]

In [0]:
import pyspark.sql.functions as F
import pandas as pd

# a. Remove trips finishing before the starting time
yellow_df_clean = yellow_df_clean[yellow_df_clean['tpep_dropoff_datetime'] > yellow_df_clean['tpep_pickup_datetime']]
green_df_clean = green_df_clean[green_df_clean['lpep_dropoff_datetime'] > green_df_clean['lpep_pickup_datetime']]

# Remove data with passenger count < 1
yellow_df_clean = yellow_df_clean.filter(yellow_df_clean['passenger_count'] >= 1)
green_df_clean = green_df_clean.filter(green_df_clean['passenger_count'] >= 1)

# Remove trips with negtive charge amount or 0 fare or total amount
yellow_df_clean = yellow_df_clean.filter(
    (yellow_df_clean['fare_amount'] > 0) &
    (yellow_df_clean['tolls_amount'] >= 0) &
    (yellow_df_clean['extra'] >= 0) &
    (yellow_df_clean['mta_tax'] >= 0) &
    (yellow_df_clean['tip_amount'] >= 0) &
    (yellow_df_clean['tolls_amount'] >= 0) &
    (yellow_df_clean['improvement_surcharge'] >= 0) &
    (yellow_df_clean['total_amount'] > 0 )&
    (yellow_df_clean['airport_fee'] >= 0 ) &
    (yellow_df_clean['congestion_surcharge'] >= 0 ) 
    )

green_df_clean = green_df_clean.filter(
    (green_df_clean['fare_amount'] > 0) &
    (green_df_clean['tolls_amount'] >= 0) &
    (green_df_clean['extra'] >= 0) &
    (green_df_clean['mta_tax'] >= 0) &
    (green_df_clean['tip_amount'] >= 0) &
    (green_df_clean['tolls_amount'] >= 0) &
    (green_df_clean['improvement_surcharge'] >= 0) &
    (green_df_clean['total_amount'] > 0 ) &
    (green_df_clean['congestion_surcharge'] >= 0)
)

In [0]:
from pyspark.sql.functions import expr

# Create a column with the trip duration in min
yellow_df_clean = yellow_df_clean.withColumn(
    "trip_duration",
    expr("round((unix_timestamp(tpep_dropoff_datetime) - unix_timestamp(tpep_pickup_datetime)) / 60, 2)")
)

green_df_clean = green_df_clean.withColumn(
    "trip_duration",
    expr("round((unix_timestamp(lpep_dropoff_datetime) - unix_timestamp(lpep_pickup_datetime)) / 60, 2)")
)

In [0]:
# Create a column speed with unit km/hour 
yellow_df_clean = yellow_df_clean.withColumn(
    "speed",
    expr("round((trip_distance * 1.60934) / (trip_duration / 60), 3)")
)

green_df_clean = green_df_clean.withColumn(
    "speed",
    expr("round((trip_distance * 1.60934) / (trip_duration / 60), 3)")
)

In [0]:

yellow_df_clean = yellow_df_clean.filter(
    (yellow_df_clean['trip_duration'] >= 1) &
    (yellow_df_clean['trip_duration'] <= 240) &
    (yellow_df_clean['trip_distance'] >= 0.2 ) &
    (yellow_df_clean['trip_distance'] <= 60 ) &
    (yellow_df_clean['speed'] > 0) &
    (yellow_df_clean['speed'] <= 55 )
)

green_df_clean = green_df_clean.filter(
    (green_df_clean['trip_duration'] >= 1) &
    (green_df_clean['trip_duration'] <= 240) &
    (green_df_clean['trip_distance'] >= 0.2 ) &
    (green_df_clean['trip_distance'] <= 60 ) &
    (green_df_clean['speed'] > 0) &
    (green_df_clean['speed'] <= 55 )
)

In [0]:
# rename 
green_df_clean = green_df_clean.withColumnRenamed("lpep_pickup_datetime", "tpep_pickup_datetime")
green_df_clean = green_df_clean.withColumnRenamed("lpep_dropoff_datetime", "tpep_dropoff_datetime")

In [0]:
from pyspark.sql.functions import lit
green_df_clean = green_df_clean.withColumn("taxi_type", lit("green"))
yellow_df_clean = yellow_df_clean.withColumn("taxi_type", lit("yellow"))

In [0]:
from pyspark.sql.types import DoubleType,IntegerType,StringType

yellow_df_clean = yellow_df_clean.withColumn("ehail_fee", lit(None).cast(DoubleType()))
yellow_df_clean = yellow_df_clean.withColumn("trip_type", lit(None).cast(DoubleType()))

# Add missing columns to cleaned_green_taxi_df and fill with nulls
green_df_clean = green_df_clean.withColumn("airport_fee", lit(None).cast(DoubleType())) 
green_df_clean = green_df_clean.withColumn("payment_type ", lit(None).cast(IntegerType()))
green_df_clean = green_df_clean.withColumn("store_and_fwd_flag", lit(None).cast(StringType()))

combined_df = yellow_df_clean.union(green_df_clean)

In [0]:
combined_df = combined_df.join(
    taxi_zone_lookup,
    on=(combined_df['PULocationID'] == taxi_zone_lookup['LocationID']),
    how='left'
)

In [0]:
combined_df = combined_df.withColumnRenamed("Borough", "pickup_Borough") \
                     .withColumnRenamed("Zone", "pickup_Zone") \
                     .withColumnRenamed("service_zone", "pickup_service_zone")
                     
combined_df = combined_df.drop('LocationID')

In [0]:
combined_df = combined_df.join(
    taxi_zone_lookup,
    on=(combined_df['DOLocationID'] == taxi_zone_lookup['LocationID']),
    how='left'
)

In [0]:
combined_df = combined_df.withColumnRenamed("Borough", "dropoff_Borough") \
                     .withColumnRenamed("Zone", "dropoff_Zone") \
                     .withColumnRenamed("service_zone", "dropoff_service_zone")
                     
combined_df = combined_df.drop('LocationID')

In [0]:
display(combined_df.limit(5))

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,trip_duration,speed,taxi_type,ehail_fee,trip_type,pickup_Borough,pickup_Zone,pickup_service_zone,dropoff_Borough,dropoff_Zone,dropoff_service_zone
1,2018-01-11T22:26:54.000+0000,2018-01-11T22:47:58.000+0000,1.0,2.1,1.0,Y,141.0,238.0,1.0,14.5,2.5,0.5,2.56,0.0,0.3,20.36,2.0,0.0,21.07,9.624,yellow,null,null,Manhattan,Lenox Hill West,Yellow Zone,Manhattan,Upper West Side North,Yellow Zone
1,2018-01-11T23:56:19.000+0000,2018-01-12T00:03:56.000+0000,1.0,1.0,1.0,Y,186.0,234.0,1.0,7.0,2.5,0.5,2.05,0.0,0.3,12.35,2.0,0.0,7.62,12.672,yellow,null,null,Manhattan,Penn Station/Madison Sq West,Yellow Zone,Manhattan,Union Sq,Yellow Zone
1,2018-01-12T00:47:17.000+0000,2018-01-12T00:55:40.000+0000,1.0,1.8,1.0,Y,239.0,43.0,1.0,8.5,2.5,0.5,2.35,0.0,0.3,14.15,2.0,0.0,8.38,20.741,yellow,null,null,Manhattan,Upper West Side South,Yellow Zone,Manhattan,Central Park,Yellow Zone
1,2018-01-12T01:06:08.000+0000,2018-01-12T01:08:21.000+0000,1.0,0.4,1.0,Y,237.0,237.0,1.0,3.5,2.5,0.5,1.7,0.0,0.3,8.5,2.0,0.0,2.22,17.398,yellow,null,null,Manhattan,Upper East Side South,Yellow Zone,Manhattan,Upper East Side South,Yellow Zone
1,2018-01-12T01:36:42.000+0000,2018-01-12T01:59:20.000+0000,1.0,4.4,1.0,Y,249.0,263.0,1.0,17.5,2.5,0.5,4.15,0.0,0.3,24.95,2.0,0.0,22.63,18.774,yellow,null,null,Manhattan,West Village,Yellow Zone,Manhattan,Yorkville West,Yellow Zone


In [0]:
combined_df.write.parquet("dbfs:/FileStore/combined_df.parquet", mode="overwrite")